# IMPORTING ALL DEPENDANCIES I NEED FOR THIS PROJECT

In [ ]:
import numpy as np
import pandas as pd
import missingno as msno 
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import statsmodels.graphics.correlation as sgc
from statsmodels.graphics.gofplots import qqplot
import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import OLSInfluence
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from geopy.geocoders import Nominatim
from time import sleep

# CONNECTION TO MY DATABASE ON POSTGRES

This is the connection link to my database on postgreSQL, the actual connection function is on the file **db_connect.py**

In [ ]:
# Import necessary packages
import pandas as pd
import sys
sys.path.append('..')  # Go up one level to the root folder
from db_connect import connect_to_db

# Step 1: Connect to the database
conn = connect_to_db()

# Step 2: Create a cursor and run a query
cursor = conn.cursor()
query = "SELECT * FROM airbnbs_nairobi.listing_data_yearly;"
cursor.execute(query)

# Step 3: Fetch results and convert to a DataFrame
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

# Step 4: Display the data
print("Connection successful! Previewing data:")
display(df.head())

In [ ]:
df

# Data Exploration an attempt at understanding my data

In [ ]:
df.info()

In [ ]:
df.describe()

# Data Cleaning

In [ ]:
import re

def to_snake_case(name):
    # Convert to lowercase
    name = name.lower()
    # Replace spaces with underscores
    name = name.replace(' ', '_')
    # Remove special characters like parentheses
    name = re.sub(r'[(%\)]+', '', name)
    # Replace multiple underscores with single underscore
    name = re.sub(r'_+', '_', name)
    return name

# Apply to all columns
df.columns = [to_snake_case(col) for col in df.columns]

print(df.columns)

In [ ]:
msno.matrix(df)

# Pricing & Revenue Analysis

## What's the correlation between listing type and average nightly rate?
For this analysis, room type is used as the primary classification variable, as it provides a more general and consistent categorization of listings. In contrast, listing type contains a large number of highly specific categories, which can introduce unnecessary complexity and reduce comparability across observations.

In [ ]:
listing_type = df['listing_type'].unique()
len(listing_type)

In [ ]:
room_type = df['room_type'].unique()
len(room_type)
room_type

While **listing types** consist of **29 unique categories** across the dataset, **room types** are limited to **three broad and representative categories** that better capture the nature of the accommodation. These include **Private Room**, **Entire Home/Apartment**, and **Hotel Room**, the latter of which appears only once in the dataset.

#### Distribution of Room Type across the entire dataset

In [ ]:
room_type_count = df['room_type'].value_counts()
plt.figure(figsize=(10,8))
plt.pie(room_type_count, labels=room_type_count.index, colors=("#0FB9B9", "#E29015", "#F3013E"), autopct='%1.1f%%', startangle=90, textprops={'fontsize':11})
plt.title("Room Type Distribution Across the Dataset", fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
listing_type_vs_avg_rate_per_night = df.groupby('room_type')['avg_rate_per_year'].mean().sort_values(ascending=False).round(2)
plt.figure(figsize=(10, 6))
listing_type_vs_avg_rate_per_night.plot(kind='bar', color=["#1AF64D", "#10EFCD", "#F60F0FF3"])
plt.title('Listing Type correlation to Average Rate Per Night', fontsize=14, fontweight='bold')
plt.ylabel('Average rate per Night', fontsize=14)
plt.xlabel('Type of Listing', fontsize=14)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# display the findings
listing_type_vs_avg_rate_per_night

Based on the analysis, **hotel rooms** command the highest average nightly rate at **KSh 15,401**. In contrast, **entire homes** and **private rooms** are priced at relatively similar levels, with average nightly rates of **KSh 7,488** and **KSh 7,424**, respectively.

## Which room types generate the most revenue despite having lower average rates?

In [ ]:
avg_rate_vs_revenue_per_room_type = df.groupby('room_type')['revenue_per_year'].mean().sort_values(ascending=False).round(2)
avg_rate_vs_revenue_per_room_type

In [ ]:
avg_rate_vs_revenue_per_room_type = (
    df.groupby('room_type')
      .agg(
          avg_rate_per_night=('avg_rate_per_year', 'mean'),
          revenue_per_year=('revenue_per_year', 'mean')
      )
).sort_values(by='revenue_per_year', ascending=False).round(2)

avg_rate_vs_revenue_per_room_type

The listing type generating the most revenue is **Entire Home, Ksh 619762** despite being the second highest rate per night, followed by **Private Room** generating **Ksh 229331** per year and lastly **Hotel Rooms** generating **Ksh 168583** despite being the highest rate per night

# Host Strategy & Management

## Are professionally managed listings priced higher than individually managed ones?

A listing is considered **professionally managed** when it is:

*   Operated by a **property management company** or hospitality firm
    
*   Managed by hosts who handle **multiple properties** as a business rather than a single personal residence
    

This typically includes:

*   Standardized check-in/check-out procedures
    
*   Dedicated cleaning and maintenance teams
    
*   Consistent pricing and availability management
    
*   Formal guest communication and support systems
    

How this differs from individual hosts

*   **Individually managed listings** are usually run by:
    
    *   A single host
        
    *   Often the property owner
        
    *   With more personalized and less standardized operations
        
*   **Professionally managed listings** tend to:
    
    *   Have **stricter policies** (e.g., cancellation, minimum nights)
        
    *   Operate more like hotels or serviced apartments
        
    *   Prioritize occupancy optimization and operational efficiency

In [ ]:
# Map boolean values to labels
management_labels = df['professional_management'].map({
    True: 'Professionally Managed',
    False: 'Individually Managed'
})

professional_management_pie = management_labels.value_counts()

colors = ("#5da718", "#79059c")

plt.figure(figsize=(10, 8))
plt.pie(
    professional_management_pie,
    labels=professional_management_pie.index,
    autopct='%1.1f%%',
    colors=colors,
    startangle=90,
    textprops={'fontsize': 11}
)
plt.title("Professionally Managed vs Individually Managed Listings")
plt.tight_layout()
plt.show()

# Display counts
print(professional_management_pie)


so 99 Listings making up 33% of all the listings are professionally managed while 201 Listings making uo 67% of all the listings are Individually managed

In [ ]:
management_labels = df['professional_management'].map({
    True: 'Professionally Managed',
    False: 'Individually Managed'
})

pricing_vs_avg_rate_per_year = df.groupby(management_labels)['avg_rate_per_year'].mean().round(2)
pricing_vs_avg_rate_per_year

In [ ]:
plt.figure(figsize=(10,6))
pricing_vs_avg_rate_per_year.plot(kind='bar', color=["#05258D","#067538"])
plt.title('Professional Management Correlation To Average Price', fontsize=12, fontweight='bold')
plt.xlabel('Professional Management', fontsize=12)
plt.xticks(rotation=0)
plt.ylabel('Average Rate Per Year', fontsize=12)
plt.tight_layout()
plt.show()


based on our chart above, the professionally managed listings tend to be priced higher compared to individually run listings, With professionally managed listings charging an average a rate of **8086 per Night** and Individually Managed Listings charging an average rate of **7219 per Night**

## Do professionally managed listings have better reviews/ratings?

In [ ]:
professionally_managed_vs_reviews = df.groupby(management_labels)['rating_overall'].mean().sort_values(ascending=True).round(2)
professionally_managed_vs_reviews

**Individually Managed** Listings are slightly better rated at **4.79** compared to **Professionally Managed** listings at **4.77**

## Is there a relationship between professional management and cancellation strictness?

To assess whether host management strategy influences booking flexibility, we examined the distribution of cancellation policies across professionally and individually managed listings.

In [ ]:
professional_management_vs_cancellation_strictness = df.groupby('cancellation_policy')['professional_management'].value_counts
professional_management_vs_cancellation_strictness

In [ ]:
pm_vs_policy = (
    pd.crosstab(
        df['cancellation_policy'],
        df['professional_management'].map({
            True: 'Professionally Managed',
            False: 'Individually Managed'
        })
    )
)


In [ ]:
pm_vs_policy.plot(
    kind='bar',
    stacked=True,
    figsize=(10, 6)
)

plt.title('Professional Management vs Cancellation Policy')
plt.xlabel('Cancellation Policy')
plt.ylabel('Number of Listings')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


### Key Insights: Professional Management vs Cancellation Policy

1.  **Flexible policies are dominated by individually managed listings**The majority of listings with _Flexible_ cancellation policies are individually managed, suggesting that solo hosts prioritize flexibility to attract bookings and remain competitive.
    
2.  **Professionally managed listings are more concentrated in stricter policies**Under _Strict_ and _Moderate_ cancellation policies, professionally managed listings make up a noticeably larger share. This indicates that professional operators are more comfortable enforcing stricter terms, likely due to better demand forecasting, operational buffers, and portfolio diversification.
    
3.  **Moderate policies represent a middle ground for both host types**Both individual and professional hosts are strongly represented under _Moderate_ policies, reinforcing the idea that this policy balances guest appeal with revenue protection.
    
4.  **Firm and Limited policies are relatively rare**Very few listings adopt _Firm_ or _Limited_ cancellation policies, suggesting low market preference—possibly due to reduced guest willingness to book under highly restrictive conditions.
    

### Strategic Interpretation

Overall, **professional management is associated with stricter cancellation strategies**, while **individual hosts rely more on flexibility to drive demand**. This highlights differing risk tolerance and pricing strategies between professional operators and independent hosts in Nairobi’s Airbnb market.

## What price range attracts the most bookings (occupancy)?


In [ ]:
# Create price bins
df['price_range'] = pd.cut(df['avg_rate_per_year'], 
                            bins=[0, 5000, 10000, 15000, 20000],
                            labels=['Budget (0-5k)', 'Mid-range (5-10k)', 'Premium (10-15k)', 'Luxury (15k+)'])

# Group by price range
best_price_metric = df.groupby('price_range')['annual_occupancy'].agg(['mean', 'count']).sort_values('mean', ascending=False)

plt.figure(figsize=(10, 6))
best_price_metric['mean'].plot(kind='bar', color='steelblue')
plt.title('Average Occupancy by Price Range', fontsize=16, fontweight='bold')
plt.xlabel('Price Range', fontsize=14)
plt.ylabel('Average Occupancy (%)', fontsize=14)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

print(best_price_metric)

### Insights: Average Occupancy by Price Range


1.  **Mid-range listings (KSh 5k–10k) attract the most customers**With the highest average occupancy, mid-range listings appear to offer the best balance between price and perceived value. This price segment is likely the most attractive to the majority of guests in Nairobi.
    
2.  **Luxury listings (KSh 15k+) maintain strong demand**Despite higher prices, luxury listings show relatively high occupancy, suggesting a consistent market for premium accommodation—possibly driven by business travelers, expatriates, or high-end tourists.
    
3.  **Budget listings (Below KSh 5k) do not achieve the highest occupancy**While budget listings are cheaper, their lower occupancy compared to mid-range listings suggests that **price alone is not the primary driver of demand**. Factors such as location, amenities, and perceived quality likely play a significant role.
    
4.  **Premium listings (KSh 10k–15k) show moderate occupancy**Premium listings fall between mid-range and budget listings in terms of occupancy, indicating that demand tapers slightly as prices rise beyond the mid-range threshold.
    

### Key Takeaway


> **Mid-range Airbnb listings (KSh 5k–10k) attract the highest customer demand, highlighting a value-for-money sweet spot in Nairobi’s short-term rental market.**

## How does cancellation policy affect pricing strategy?

In this segment, we check the various types of cancellation policies present within our dataset

In [ ]:
plt.figure(figsize=(10, 8))
cancellation_policy_count = df['cancellation_policy'].value_counts()
colors = ['#FF6B6B', "#6F0DD0", '#45B7D1', "#EDC914", "#4EED14"]
plt.pie(cancellation_policy_count, labels=cancellation_policy_count.index, autopct='%1.1f%%', 
        colors=colors, startangle=90, textprops={'fontsize': 11})
plt.title("Cancellation Policy Distribution", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Display the counts
print(cancellation_policy_count)

### Cancellation Policy Distribution

The dataset is dominated by **Flexible** and **Moderate** cancellation policies, with **43.7%** and **36.7%** listings respectively. This suggests that most hosts prefer policies that allow guests greater freedom to cancel reservations with minimal penalties.

**Firm** and **Strict** policies are less common, appearing in **11.3%** and **8.0%** listings respectively, indicating a smaller segment of hosts who prioritize booking certainty over flexibility.

Only **one** listing follows a **Limited** cancellation policy, making it negligible in the overall distribution.

### Key Insight

Overall, the distribution reflects a **guest-friendly marketplace**, where flexible cancellation options are more prevalent than restrictive policies, likely aimed at attracting short-term and undecided travelers.

In [ ]:
valid_policies = ['Flexible', 'Moderate', 'Firm', 'Strict']
cancellation_policy_vs_pricing = df[df['cancellation_policy'].isin(valid_policies)].groupby('cancellation_policy')['avg_rate_per_year'].mean().sort_values(ascending=False).round(2)
cancellation_policy_vs_pricing

so i got rid of **Limited** since it's only one entry, i don't think it a good representation of the insights i'd like to derive from the dataset

In [ ]:
plt.figure(figsize=(10,6))
cancellation_policy_vs_pricing.plot(kind='bar')
plt.title('Cancellation Policy and it\'s effect on Pricing of the Listing',fontsize=14,fontweight='bold' )
plt.ylabel('Average Rate per Night', fontsize=12)
plt.xlabel('Cancellation Policy', fontsize=12)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


### Effect of Cancellation Policy on Listing Pricing

The chart shows a **clear relationship between cancellation strictness and average nightly price**.

*   **Moderate cancellation policies** have the **highest average nightly rates**, suggesting that hosts with mid-level flexibility are able to charge a premium—possibly balancing guest trust with revenue protection.
    
*   **Strict policies** follow closely, indicating that listings enforcing tighter cancellation rules still command relatively high prices, likely due to stronger demand, desirable locations, or higher-quality listings.
    
*   **Firm policies** sit in the middle range, reflecting a moderate pricing strategy.
    
*   **Flexible cancellation policies** are associated with the **lowest average nightly rates**, suggesting that hosts may lower prices to compensate for the higher risk of last-minute cancellations.
    

###  Interpretation

Overall, **less flexible cancellation policies tend to correlate with higher pricing**, implying that hosts who restrict cancellations may do so confidently when their listings have strong market appeal. Conversely, greater flexibility appears to be used as a competitive pricing lever to attract more bookings.

#  Occupancy & Booking Patterns

## What's the average occupancy rate across the dataset?
The average occupancy rate across all active Airbnb listings in Nairobi provides a baseline measure of market demand. This metric helps contextualize how different pricing strategies, room types, and cancellation policies perform relative to the overall market.

In [ ]:
average_occupancy_rate = df['annual_occupancy'].mean().round(2)
average_occupancy_rate

Average occupancy rate across the dataset is **23.39%**

## Do flexible cancellation policies lead to higher occupancy?

In [ ]:
valid_policies = ['Flexible', 'Moderate', 'Firm', 'Strict']
cancellation_policy_vs_occupancy = df[df['cancellation_policy'].isin(valid_policies)].groupby('cancellation_policy')['annual_occupancy'].mean().sort_values(ascending=False).round(2)
cancellation_policy_vs_occupancy

From our Findings,Listings with  **Strict** cancellation policy holds the largest percentage of annual occupancy rate at **26.35%**, Listings with **Moderate** cancellation policy come in second with **23.61%**, Firm Cancellation policy listings hold a **23.29%** annual occupancy rate and listings with **Flexible** cancellation Policy hold the least annual occupancy percentage at **22.42%**. So to answer the question, **No** 
#### Flexible Cancellation policies do not attract higher annual occupancy

## How does minimum night requirement affect booking frequency?

In [ ]:
minimum_nights_vs_booking_frequency = df.groupby('minimum_nights')['annual_occupancy'].mean().sort_values(ascending=False)
minimum_nights_vs_booking_frequency

In [ ]:
plt.figure(figsize=(10,6))
minimum_nights_vs_booking_frequency.plot(kind='barh')
plt.title('Minimum Nights vs Booking Frequency', fontsize=14, fontweight='bold')
plt.ylabel('Minimum Nights', fontsize=12)
plt.xlabel('Annual Occupancy Percentage', fontsize=12)
plt.tight_layout()
plt.show()

**1\. Long-Term Stays Dominate Annual Occupancy**

*   **30-Day Stays are King:** The most significant finding is that bookings with a minimum stay of **30 nights** account for the highest annual occupancy percentage by a wide margin, reaching approximately **82%**.
    
*   **Strong Performance of 28-Day Stays:** The second-highest occupancy is for **28-night** minimum stays, at around **44%**.
    
*   **Conclusion:** This suggests a very strong market for monthly or near-monthly rentals, which could be driven by corporate housing, digital nomads, or temporary relocations. These long stays are the primary drivers of occupancy in this dataset.
    

**2\. Weekly and Long-Weekend Stays are Important Secondary Drivers**

*   **The "Sweet Spot" for Shorter Stays:** Minimum stays of **7 nights** and **3 nights** have very similar and significant occupancy percentages, both hovering around the **30-31%** mark.
    
*   **Conclusion:** There is substantial demand for weekly vacations and extended weekend trips. For hosts not targeting the monthly market, these two durations appear to be the most effective for maintaining occupancy.
    

**3\. Very Short Stays Contribute Less to Overall Occupancy**

*   **1-2 Night Stays:** While extremely common in the short-term rental market, minimum stays of **2 nights** (~26%) and **1 night** (~20%) contribute less to the total annual occupancy than the 3, 7, 28, and 30-night options.
    
*   **Implication:** This could indicate that while these bookings may be frequent, the higher turnover results in more unbooked "gap days," leading to a lower overall occupancy percentage over the course of a year compared to longer, more continuous stays.
    

**4\. Specific Durations are Less Effective**

*   **Low Occupancy for 4, 5, and 14 Nights:** Minimum stays of **4 nights** (~14%), **14 nights** (~4%), and **5 nights** (~3%) show the lowest annual occupancy percentages.
    
*   **Conclusion:** These specific booking windows seem to be less popular with guests or less successfully utilized by hosts compared to the standard 1-night, weekend (2-3 nights), weekly (7 nights), or monthly models.
    

**Strategic Implication for Hosts:** To maximize annual occupancy, the data suggests targeting the **30+ day market** is the most effective strategy. If that is not feasible, focusing on **7-night (weekly)** or **3-night (long weekend)** minimums would be better than setting minimums of 1, 2, 4, 5, or 14 nights.

**Key Insight** The data suggests that decreasing booking frequency (by raising minimum nights to 28+) actually increases your overall business performance (occupancy).

## Which listing types have the highest occupancy rates?

In [ ]:
listing_types_vs_occupancy_rate = df.groupby('room_type')['annual_occupancy'].mean().sort_values(ascending=False).round(2)
listing_types_vs_occupancy_rate

**Entire Homes** are the most common listing type to be booked at a rate of **25.05%**, followed by **Private Room** at a rate of **15.08%** and lastly**Hotel Rooms** making up **3.0%** of the total occupancy rate

## How many listings per host on average?

In [ ]:
no_of_hosts = df['host_id'].unique()
len(no_of_hosts)

Our dataset has 205 unique hosts all who own either one or a bunch of listings within Nairobi

In [ ]:
no_of_listing_per_host = (
    df.groupby('host_id')
      .agg(
          host_name=('host_name', 'first'),
          number_of_listings=('listing_id', 'count')
      ).sort_values(by='number_of_listings',ascending=False)
)

no_of_listing_per_host

In [ ]:
top_host = no_of_listing_per_host.iloc[0]
print(f"Top host: {top_host['host_name']} with {top_host['number_of_listings']} listings")

In [ ]:
print(f"Total hosts: {len(no_of_listing_per_host)}")
print(f"Average listings per host: {no_of_listing_per_host['number_of_listings'].mean():.2f}")
print(f"Max listings by one host: {no_of_listing_per_host['number_of_listings'].max()}")
print(f"Min listings by one host: {no_of_listing_per_host['number_of_listings'].min()}")

The dataset contains **205 unique hosts**, with an **average of 1.46 listings per host**, indicating that the market is largely composed of small-scale hosts. Most hosts operate **a single listing**, as reflected by the minimum of one listing per host. However, there is evidence of **professional or portfolio-style hosting**, with the largest host, **Samra Apartments**, managing **19 listings**. This highlights a market structure dominated by individual hosts alongside a small number of high-volume operators.

# Geographic/Neighborhood Patterns

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(df['longitude'], df['latitude'], c=df['avg_rate_per_year'], alpha=0.6)
plt.colorbar(label='Average Nightly Rate')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Pricing Distribution Across Nairobi')
plt.show()

In [ ]:
NAIROBI_BOUNDS = {
    "lat_min": -1.5,
    "lat_max": -1.1,
    "lon_min": 36.6,
    "lon_max": 37.1
}


In [ ]:
df_nairobi = df[
    (df['latitude'] >= NAIROBI_BOUNDS['lat_min']) &
    (df['latitude'] <= NAIROBI_BOUNDS['lat_max']) &
    (df['longitude'] >= NAIROBI_BOUNDS['lon_min']) &
    (df['longitude'] <= NAIROBI_BOUNDS['lon_max'])
].copy()

print(f"Listings before: {len(df)}")
print(f"Listings in Nairobi: {len(df_nairobi)}")


In [ ]:
import folium
from IPython.display import display

m = folium.Map(
    location=[-1.286389, 36.817223],  # Nairobi CBD
    zoom_start=12,
    min_zoom=11,
    max_zoom=16,
    max_bounds=True,
    bounds=NAIROBI_BOUNDS,
    tiles="OpenStreetMap"
)


In [ ]:
#m.fit_bounds(NAIROBI_BOUNDS)

In [ ]:
for row in df_nairobi.itertuples():
    if row.avg_rate_per_year < 5000:
        color = '#2EC4B6'   # Budget
    elif row.avg_rate_per_year < 10000:
        color = '#1F77B4'   # Mid-range
    elif row.avg_rate_per_year < 15000:
        color = '#FF9F1C'   # Premium
    else:
        color = '#E63946'   # Luxury

    # Create popup with more details
    popup_text = f"""
    <b style='font-size: 14px'>{row.listing_name}</b><br>
    <b>Type:</b> {row.listing_type}<br>
    <b>Room:</b> {row.room_type}<br>
    <b>Rating:</b> {row.rating_overall}/5.0<br>
    <b>Price:</b> KSh {row.avg_rate_per_year:.0f}/night
    """
    
    folium.CircleMarker(
        location=[row.latitude, row.longitude],
        radius=4,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m)

display(m)

In [ ]:

def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True)
        if location:
            address = location.raw.get('address', {})
            return pd.Series({
                'neighbourhood': address.get('neighbourhood'),
                'suburb': address.get('suburb'),
                'county': address.get('county'),
                'city': address.get('city'),
                'postcode': address.get('postcode')
            })
    except Exception:
        pass

    # ALWAYS return the same indexed Series
    return pd.Series({
        'neighbourhood': None,
        'suburb': None,
        'county': None,
        'city': None,
        'postcode': None
    })



In [ ]:
geo_cols = ['neighbourhood', 'suburb', 'county', 'city', 'postcode']

geo_df = df[['latitude', 'longitude']].apply(
    lambda x: reverse_geocode(x.latitude, x.longitude),
    axis=1
)

df[geo_cols] = geo_df

After executing the geocoding process, five new location-based columns were added to the dataset: **neighbourhood**, **suburb**, **county**, **city**, and **postcode**. While all five fields contain some missing values, **suburb** stands out as the most reliable geographic indicator, with only a single missing entry and **26 distinct areas** represented. As a result, suburb is used as the primary geographic dimension for analyzing spatial patterns within the dataset.

## Are there neighborhood clusters with similar pricing?

In [ ]:
suburbs = df['suburb'].unique()
suburbs

#### STEP 1: Aggregate pricing by suburb

In [ ]:
neighbourhoods_vs_pricing = df.groupby('suburb')['avg_rate_per_year'].mean().sort_values(ascending=True).round(2).reset_index()
neighbourhoods_vs_pricing

#### STEP 2: Prepare data for clustering

In [ ]:
X = neighbourhoods_vs_pricing[['avg_rate_per_year']]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#### STEP 3: Apply KMeans clustering

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
neighbourhoods_vs_pricing['price_cluster'] = kmeans.fit_predict(X_scaled)


#### STEP 4: Inspect the clusters

In [ ]:
neighbourhoods_vs_pricing.sort_values('avg_rate_per_year')

#### STEP 5: Visualization

In [ ]:
sns.barplot(
    data=neighbourhoods_vs_pricing,
    x='price_cluster',
    y='avg_rate_per_year'
)
plt.title("Average Nightly Rate by Neighborhood Price Cluster")
plt.show()


*   **Cluster 0 (Economy/Budget):** This category contains the highest volume of listings, representing the lower-tier accommodations.
    
*   **Cluster 2 (Mid-range):** These listings represent the middle tier of the market.
    
*   **Cluster 1 (High-end/Outlier):** This cluster contains only a single listing, suggesting it is unique or an outlier."

## Do certain neighborhoods favor specific room types?

#### Create a neighborhood × room type distribution

In [ ]:
neighborhood_room_type = (
    df
    .groupby(['suburb', 'room_type'])
    .size()
    .reset_index(name='count')
)

In [ ]:
room_type_pivot = neighborhood_room_type.pivot(
    index='suburb',
    columns='room_type',
    values='count'
).fillna(0)
room_type_pivot

In [ ]:
room_type_pivot_sorted = (
    room_type_pivot
    .assign(total=room_type_pivot.sum(axis=1))
    .sort_values('total')
    .drop(columns='total')
)


In [ ]:
room_type_pivot_sorted.plot(
    kind='barh',
    stacked=True,
    figsize=(12, 10)
)
plt.title("Room Type Distribution by Neighborhood")
plt.xlabel("Number of Listings")
plt.ylabel("Suburb")
plt.tight_layout()
plt.show()


Neighborhood × Room Type Insights
---------------------------------

### 1\. **Strong geographic concentration**

*   **Kilimani division** overwhelmingly dominates the market, with the highest number of listings by a wide margin.
    
*   **Highridge division** follows as a distant second.
    
*   All other neighborhoods have relatively **low listing density**, many with fewer than 10 listings.
    

**Insight:** Airbnb supply in Nairobi is highly concentrated in a few prime neighborhoods rather than evenly distributed.

### 2\. **Entire homes dominate across neighborhoods**

*   **Entire homes** make up the majority of listings in almost every suburb.
    
*   This dominance is most pronounced in **Kilimani** and **Highridge**, indicating strong demand for full-unit stays in these areas.
    

**Interpretation:** These neighborhoods likely attract:

*   Longer stays
    
*   Families, business travelers, or groups
    
*   Guests prioritizing privacy and space
    

### 3\. **Private rooms play a secondary role**

*   **Private rooms** appear mainly in:
    
    *   Kilimani
        
    *   Highridge
        
    *   Karen-related areas
        
*   Their presence is limited elsewhere.
    

**Interpretation:** Private rooms serve as a **price-sensitive alternative** in high-demand areas but are not a primary offering city-wide.

### 4\. **Hotel rooms are extremely rare**

*   **Hotel room listings are almost nonexistent**, appearing only marginally in a few neighborhoods.
    

**Insight:** Airbnb supply in this dataset is driven by **individual hosts and residential properties**, not traditional hospitality players.

### 5\. **Neighborhood character matters**

*   **Karen ward** shows lower volume but a mix that leans toward **entire homes**, consistent with its low-density, high-end residential profile.
    
*   Peripheral neighborhoods show **minimal diversification**, often only one room type.

## Which areas have the highest revenue potential?

In [ ]:
area_revenue = (
    df
    .groupby('suburb')
    .agg(
        avg_annual_revenue=('revenue_per_year', 'mean'),
        avg_nightly_rate=('avg_rate_per_year', 'mean'),
        avg_occupancy=('annual_occupancy', 'mean'),
        listings_count=('listing_id', 'count')
    )
    .sort_values('avg_annual_revenue', ascending=False)
    .round(2)
)

area_revenue.head(10)


In [ ]:
area_revenue.head(10)['avg_annual_revenue'] \
    .sort_values() \
    .plot(kind='barh', figsize=(10, 6))

plt.title("Top 10 Areas by Average Annual Revenue")
plt.xlabel("Average Annual Revenue (KSh)")
plt.ylabel("Suburb")
plt.show()


#### **1\. The Luxury Tier: Karen’s "High-Value" Dominance**

*   **Performance:** **Karen Ward** is the undisputed market leader, generating nearly **KSh 2,000,000** annually—more than double the next highest contender.
    
*   **The Strategy:** This dominance is driven by high **Average Daily Rates (ADR)** rather than sheer volume. These listings (villas, large homes) cater to the "Long Stay" market (diplomats, expats) where a single booking can secure months of revenue.
    
*   **Verdict:** High entry cost, but unrivaled revenue per booking.
    

#### **2\. The Efficiency Tier: The Hidden Gems (Tassia & Nairobi West)**

*   **Performance:** Surprisingly, lower-middle-income areas like **Nairobi West** (~KSh 700k) and **Tassia** (~KSh 680k) significantly outperform premium hubs.
    
*   **The Strategy:** These areas thrive on a **Volume/Occupancy** model. With lower nightly rates but consistent local demand (traders, short-term work stays), they avoid the "vacancy gaps" that plague seasonal tourist spots.
    
*   **Verdict:** Lower entry cost with higher yield efficiency.
    

#### **3\. The Saturated Tier: The "Popularity Trap" (Kilimani & Highridge)**

*   **Performance:** despite being the most famous rental hubs, **Kilimani** and **Highridge** appear in the bottom half of the top 10 (averaging ~KSh 500k).
    
*   **The Strategy:** These areas suffer from **Oversupply**. The sheer number of apartments creates fierce price competition, diluting the _average_ revenue per host. While top-tier units earn well, the "average" unit struggles to match the returns of less saturated areas.
    
*   **Verdict:** High demand is offset by extreme competition; success here requires a standout product.

# Quality & Features

## How many amenities do high-revenue listings typically have?

#### Step 1: Define high-revenue listings

In [ ]:
# Define high-revenue threshold
high_revenue_threshold = df['revenue_per_year'].quantile(0.75)

df['high_revenue'] = (
    df['revenue_per_year'] >= high_revenue_threshold
)

#### Step 2: Convert amenities into a count

In [ ]:
# Create amenities count
df['amenities_count'] = (
    df['amenities']
    .fillna('')
    .apply(lambda x: len(str(x).split(',')))
)


#### Step 3: Compare high vs non-high revenue listings

In [ ]:
amenities_vs_revenue = (
    df
    .groupby('high_revenue')['amenities_count']
    .agg(['mean', 'median', 'min', 'max', 'count'])
    .round(1)
)

amenities_vs_revenue


#### Step 4: Visualization

In [ ]:
sns.boxplot(
    data=df,
    x='high_revenue',
    y='amenities_count'
)

plt.xticks([0, 1], ['Lower Revenue', 'High Revenue'])
plt.title("Amenities Count vs Revenue Tier")
plt.ylabel("Number of Amenities")
plt.xlabel("")
plt.show()

From the box plot, **high-revenue listings typically have about** _**35–40 amenities**_.

More precisely:

*   The **median** (the line inside the box) is around **36–37 amenities**
    
*   Most high-revenue listings (the interquartile range) fall roughly between **30 and 42 amenities**
    

So a “typical” high-revenue listing offers **mid-to-high 30s in amenities**, slightly more than lower-revenue listings.

## What's the relationship between listing name length and bookings?

#### Step 1: Define the variables

In [ ]:
df['listing_name_length'] = (
    df['listing_name']
    .fillna('')
    .str.len()
)


#### Step 2: Measure correlation

In [ ]:
correlation = df['listing_name_length'].corr(
    df['reserved_days_in_year']
)

correlation

#### Weak Positive

In [ ]:
df['name_length_bin'] = pd.qcut(
    df['listing_name_length'],
    q=4,
    labels=['Short', 'Medium', 'Long', 'Very Long']
)

df.groupby('name_length_bin')['reserved_days_in_year'].mean().sort_values(ascending=False)


#### Step 3: Visualize the relationship

In [ ]:
sns.scatterplot(
    data=df,
    x='listing_name_length',
    y='reserved_days_in_year',
    alpha=0.6
)

sns.regplot(
    data=df,
    x='listing_name_length',
    y='reserved_days_in_year',
    scatter=False,
    color='red'
)

plt.title("Listing Name Length vs Booked Nights per Year")
plt.xlabel("Listing Name Length (characters)")
plt.ylabel("Booked Nights per Year")
plt.show()


### Direction of the relationship

*   The **red regression line slopes slightly upward**, indicating a **weak positive relationship** between listing name length and booked nights per year.
    
*   This means that, _on average_, listings with longer names tend to receive **slightly more bookings**, but the effect is **not strong**.
    

### Strength of the relationship

*   The points are **widely scattered** around the line.
    
*   This high dispersion shows that **listing name length alone explains very little** of the variation in bookings.
    
*   Many listings with long names still have low bookings, and some short names perform very well.
    

**Conclusion:** The relationship exists, but it is **weak**.

### Practical interpretation

*   Longer names likely help by:
    
    *   Including keywords (location, amenities, “near CBD”, “luxury”, etc.)
        
    *   Improving clarity and search visibility
        
*   However, bookings are **far more influenced** by:
    
    *   Location
        
    *   Price
        
    *   Reviews
        
    *   Amenities
        
    *   Professional management

In [ ]:
df

# Market Opportunities

## What's underrepresented in the market (low supply, high demand)?

#### Create price bands

In [ ]:
price_bins = [0, 3000, 6000, 10000, 15000, df['avg_rate_per_year'].max()]
price_labels = [
    'Low (0–3k)',
    'Lower-Mid (3k–6k)',
    'Mid (6k–10k)',
    'Upper-Mid (10k–15k)',
    'Premium (15k+)'
]

df['price_band'] = pd.cut(
    df['avg_rate_per_year'],
    bins=price_bins,
    labels=price_labels,
    include_lowest=True
)


#### Define supply and demand

In [ ]:
market_summary = (
    df.groupby(['room_type', 'price_band'])
      .agg(
          listings_count=('listing_id', 'count'),
          avg_occupancy=('annual_occupancy', 'mean'),
          avg_revenue=('revenue_per_year', 'mean')
      )
      .reset_index()
)


#### Establish “low supply” and “high demand” thresholds

In [ ]:
supply_threshold = market_summary['listings_count'].median()
occupancy_threshold = market_summary['avg_occupancy'].median()
revenue_threshold = market_summary['avg_revenue'].median()


#### Flag underrepresented segments

In [ ]:
market_summary['underrepresented'] = (
    (market_summary['listings_count'] < supply_threshold) &
    (
        (market_summary['avg_occupancy'] > occupancy_threshold) |
        (market_summary['avg_revenue'] > revenue_threshold)
    )
)

underrepresented_segments = market_summary[
    market_summary['underrepresented']
].sort_values(['avg_occupancy', 'avg_revenue'], ascending=False)

underrepresented_segments


## Are there price gaps where new listings could compete?

#### Measure supply per price band

In [ ]:
supply_by_price = (
    df.groupby('price_band')
      .agg(
          listings_count=('listing_id', 'count'),
          avg_price=('avg_rate_per_year', 'mean')
      )
      .reset_index()
)


#### Measure demand/performance per price band

In [ ]:
performance_by_price = (
    df.groupby('price_band')
      .agg(
          avg_occupancy=('annual_occupancy', 'mean'),
          avg_revenue=('revenue_per_year', 'mean')
      )
      .reset_index()
)


#### Combine supply + performance

In [ ]:
price_gap_analysis = supply_by_price.merge(
    performance_by_price,
    on='price_band'
)

price_gap_analysis.sort_values('listings_count')


#### Identify price gaps

In [ ]:
price_gap_analysis['potential_gap'] = (
    (price_gap_analysis['listings_count'] < price_gap_analysis['listings_count'].median()) &
    (price_gap_analysis['avg_occupancy'] > price_gap_analysis['avg_occupancy'].median())
)

price_gap_analysis


The analysis shows that demand is relatively consistent across all price bands, with average occupancy remaining stable regardless of nightly rate. This suggests that higher prices do not significantly deter bookings in this market.

Supply, however, is unevenly distributed. Most listings are concentrated in the low-to-mid price ranges (below 10k), indicating strong competition and limited room for new entrants to differentiate. In contrast, the upper-mid (10k–15k) and premium (15k+) segments have significantly fewer listings.

The premium segment stands out as a clear opportunity: despite the lowest supply, it records strong occupancy and the highest average revenue. The upper-mid segment also shows potential, as low supply combined with relatively lower performance suggests pricing or positioning inefficiencies that new, well-designed listings could exploit.

Overall, meaningful price gaps exist at the higher end of the market, while the lower and mid ranges appear saturated.

## Which listing types have declining revenue trends?

Revenue trend analysis typically requires time-series data. As this dataset provides a snapshot of listing performance rather than longitudinal observations, it does not support direct analysis of declining or increasing revenue trends by listing type. Instead, the analysis focuses on comparing current revenue performance across listing categories.